In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm 
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()
import multiprocessing
nCPU = multiprocessing.cpu_count()
from datetime import datetime, timedelta

/Users/koheiyamamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
ct_neighbor = pd.read_csv('CensusTractsNeighborhood.csv')
ct_neighbor = ct_neighbor.groupby('src_boro_c').nbr_boro_c.apply(list)
ct_neighbor = ct_neighbor.to_frame()
ct_neighbor.reset_index(inplace=True, drop=False)
ct_neighbor

,src_boro_c,nbr_boro_c
0,1000201,"[1000202, 1000600, 1001401]"
1,1000202,"[1000201, 1001001, 1001200, 1001401, 3002100]"
2,1000500,"[3000700, 3004700, 3005300]"
3,1000600,"[1000201, 1000800, 1001401, 1001600]"
4,1000700,"[1000900, 1001300, 1001502, 3000100, 3000301]"
...,...,...
2157,5030301,"[5023900, 5024700, 5025100, 5029102, 5029104, ..."
2158,5030302,"[5023100, 5023900, 5030301, 5031901, 5032300]"
2159,5031901,"[5022300, 5023100, 5030302, 5031902, 5032300]"
2160,5031902,"[5022300, 5023100, 5031901, 5032300]"


In [3]:
def add_dt_1(data):
    return datetime.strptime(str(data.stop_frisk1_XYTableToPoint_S_2) + ' ' + str(data.stop_frisk1_XYTableToPoint_S_3), '%Y-%m-%d %H:%M:%S')
        
frisk = pd.read_csv('stop_frisk_census_near_raw.csv')
frisk.dropna(inplace=True, how='any')
frisk.drop_duplicates(subset=['stop_frisk1_XYTableToPoint_S_2', 'stop_frisk1_XYTableToPoint_S_3', 'geo_export_267875c8_5ee9_4cb_13'], inplace=True)
frisk['dt'] = frisk.apply(add_dt_1, axis=1)
frisk['ct'] = frisk['geo_export_267875c8_5ee9_4cb_13'].apply(lambda x: int(x))
frisk.drop(['stop_frisk1_XYTableToPoint_S_2', 'stop_frisk1_XYTableToPoint_S_3', 'OBJECTID', 'geo_export_267875c8_5ee9_4cb_13'], axis='columns', inplace=True)
frisk

,dt,ct
0,2017-01-16 14:26:00,1003300
2,2017-02-08 11:10:00,1001300
3,2017-02-20 11:35:00,1004900
4,2017-02-21 13:20:00,1004900
5,2017-02-17 21:25:00,4142900
...,...,...
11614,2017-10-10 17:40:00,1016400
11615,2017-10-12 16:05:00,1016400
11616,2017-01-03 19:41:00,2021700
11617,2017-01-08 23:10:00,3034800


In [4]:
def add_dt_2(data):
    return datetime.strptime(str(data.Date) + ' ' + str(data.Time), '%m/%d/%Y %H:%M:%S')

crimes = pd.concat(([pd.read_csv('personal_near_sorted.csv'), pd.read_csv('property_near_sorted.csv')]))
crimes.reset_index(inplace=True, drop=True)
crimes.dropna(inplace=True, how='any')
crimes.isnull().sum()
crimes['dt'] = crimes.apply(add_dt_2, axis=1)
crimes

,boro_ct2010,Date,Time,Type,dt
0,4011200,1/1/2017,0:01:00,PersonalCrime,2017-01-01 00:01:00
1,2014100,1/1/2017,0:01:00,PersonalCrime,2017-01-01 00:01:00
2,2004600,1/1/2017,0:01:00,PersonalCrime,2017-01-01 00:01:00
3,4033402,1/1/2017,0:01:00,PersonalCrime,2017-01-01 00:01:00
4,3038700,1/1/2017,18:00:00,PersonalCrime,2017-01-01 18:00:00
...,...,...,...,...,...
141082,2042400,6/2/2017,15:45:00,PropertyCrime,2017-06-02 15:45:00
141083,3051700,8/11/2017,20:15:00,PropertyCrime,2017-08-11 20:15:00
141084,1031703,9/26/2017,12:17:00,PropertyCrime,2017-09-26 12:17:00
141085,3052700,2/15/2017,16:00:00,PropertyCrime,2017-02-15 16:00:00


In [ ]:
def get_frisk_counts__id(_dt, __ct):
    df_ct = frisk.query('ct == @__ct')
    c = 0
    for i in df_ct.itertuples():
        td_diff = (_dt - i.dt).total_seconds()
        if 0 < td_diff and td_diff < 21600: # 6hrs
            c = c + 1
            print(_dt, '-', i.dt, '=', td_diff)
    return c

def get_frisk_counts__neighbors(_dt, _ct):
    if ct_neighbor.query('src_boro_c == @_ct').empty:
        return 0
    else:
        neighbors = ct_neighbor.query('src_boro_c == @_ct').iloc[0, 1]
        c_sum = 0
        for neighbor in neighbors:
            c_sum = c_sum + get_frisk_counts__id(_dt, neighbor)
        return c_sum

counts_ct_list = []
counts_neighbors_list = []
counts_total_list = []
for crime in tqdm(crimes.itertuples()):
    counts_ct = get_frisk_counts__id(crime.dt, crime.boro_ct2010)
    counts_neighbors = get_frisk_counts__neighbors(crime.dt, crime.boro_ct2010)
    counts_ct_list.append(counts_ct)
    counts_neighbors_list.append(counts_neighbors)
    counts_total_list.append(counts_ct + counts_neighbors)
#     print(counts_ct, counts_neighbors, crime)

2017-01-01 03:30:00 - 2017-01-01 02:00:00 = 5400.0
2017-01-01 13:20:00 - 2017-01-01 09:00:00 = 15600.0
2017-01-02 19:30:00 - 2017-01-02 16:51:00 = 9540.0
2017-01-02 02:10:00 - 2017-01-01 22:50:00 = 12000.0
2017-01-02 00:05:00 - 2017-01-01 22:58:00 = 4020.0
2017-01-02 02:10:00 - 2017-01-01 22:50:00 = 12000.0
2017-01-03 00:01:00 - 2017-01-02 19:17:00 = 17040.0
2017-01-03 18:00:00 - 2017-01-03 17:45:00 = 900.0
2017-01-03 18:03:00 - 2017-01-03 17:10:00 = 3180.0
2017-01-03 18:00:00 - 2017-01-03 17:45:00 = 900.0
2017-01-04 21:00:00 - 2017-01-04 16:54:00 = 14760.0
2017-01-04 20:40:00 - 2017-01-04 14:44:00 = 21360.0
2017-01-04 22:46:00 - 2017-01-04 18:33:00 = 15180.0
2017-01-04 19:40:00 - 2017-01-04 15:55:00 = 13500.0
2017-01-04 03:00:00 - 2017-01-04 02:20:00 = 2400.0
2017-01-04 14:20:00 - 2017-01-04 11:10:00 = 11400.0
2017-01-04 23:00:00 - 2017-01-04 18:33:00 = 16020.0
2017-01-04 17:30:00 - 2017-01-04 17:00:00 = 1800.0
2017-01-04 19:00:00 - 2017-01-04 15:55:00 = 11100.0
2017-01-05 23:20:00 - 

2017-02-04 19:40:00 - 2017-02-04 16:53:00 = 10020.0
2017-02-04 19:40:00 - 2017-02-04 19:20:00 = 1200.0
2017-02-04 01:15:00 - 2017-02-03 21:53:00 = 12120.0
2017-02-04 01:15:00 - 2017-02-03 22:51:00 = 8640.0
2017-02-04 23:00:00 - 2017-02-04 20:30:00 = 9000.0
2017-02-05 13:40:00 - 2017-02-05 09:58:00 = 13320.0
2017-02-05 07:50:00 - 2017-02-05 04:45:00 = 11100.0
2017-02-05 18:30:00 - 2017-02-05 17:02:00 = 5280.0
2017-02-05 21:00:00 - 2017-02-05 16:55:00 = 14700.0
2017-02-05 17:00:00 - 2017-02-05 16:55:00 = 300.0
2017-02-05 03:30:00 - 2017-02-05 03:10:00 = 1200.0
2017-02-06 09:00:00 - 2017-02-06 03:46:00 = 18840.0
2017-02-06 14:00:00 - 2017-02-06 10:10:00 = 13800.0
2017-02-06 21:00:00 - 2017-02-06 19:35:00 = 5100.0
2017-02-06 06:15:00 - 2017-02-06 03:46:00 = 8940.0
2017-02-07 22:10:00 - 2017-02-07 20:30:00 = 6000.0
2017-02-07 22:10:00 - 2017-02-07 17:20:00 = 17400.0
2017-02-07 19:49:00 - 2017-02-07 16:38:00 = 11460.0
2017-02-07 21:00:00 - 2017-02-07 16:56:00 = 14640.0
2017-02-07 17:00:00 - 

2017-03-09 02:55:00 - 2017-03-09 02:47:00 = 480.0
2017-03-10 20:58:00 - 2017-03-10 19:10:00 = 6480.0
2017-03-11 07:05:00 - 2017-03-11 04:20:00 = 9900.0
2017-03-11 06:30:00 - 2017-03-11 02:00:00 = 16200.0
2017-03-11 01:00:00 - 2017-03-10 22:15:00 = 9900.0
2017-03-11 06:30:00 - 2017-03-11 02:00:00 = 16200.0
2017-03-12 17:00:00 - 2017-03-12 14:20:00 = 9600.0
2017-03-12 00:01:00 - 2017-03-11 20:11:00 = 13800.0
2017-03-12 11:45:00 - 2017-03-12 10:17:00 = 5280.0
2017-03-12 21:25:00 - 2017-03-12 17:00:00 = 15900.0
2017-03-12 17:00:00 - 2017-03-12 14:20:00 = 9600.0
2017-03-12 12:08:00 - 2017-03-12 11:12:00 = 3360.0
2017-03-12 17:00:00 - 2017-03-12 14:20:00 = 9600.0
2017-03-12 15:54:00 - 2017-03-12 14:20:00 = 5640.0
2017-03-12 20:00:00 - 2017-03-12 14:07:00 = 21180.0
2017-03-12 11:30:00 - 2017-03-12 09:22:00 = 7680.0
2017-03-12 17:00:00 - 2017-03-12 14:20:00 = 9600.0
2017-03-12 00:08:00 - 2017-03-11 22:15:00 = 6780.0
2017-03-13 17:35:00 - 2017-03-13 15:15:00 = 8400.0
2017-03-14 20:05:00 - 2017-

2017-04-14 15:00:00 - 2017-04-14 10:02:00 = 17880.0
2017-04-14 19:45:00 - 2017-04-14 15:20:00 = 15900.0
2017-04-14 15:00:00 - 2017-04-14 10:02:00 = 17880.0
2017-04-14 02:15:00 - 2017-04-13 22:10:00 = 14700.0
2017-04-14 18:40:00 - 2017-04-14 15:55:00 = 9900.0
2017-04-14 03:25:00 - 2017-04-13 23:30:00 = 14100.0
2017-04-14 15:03:00 - 2017-04-14 10:02:00 = 18060.0
2017-04-14 19:20:00 - 2017-04-14 15:20:00 = 14400.0
2017-04-15 04:09:00 - 2017-04-15 01:21:00 = 10080.0
2017-04-15 16:15:00 - 2017-04-15 12:39:00 = 12960.0
2017-04-15 00:00:00 - 2017-04-14 23:07:00 = 3180.0
2017-04-15 20:50:00 - 2017-04-15 17:27:00 = 12180.0
2017-04-16 03:30:00 - 2017-04-16 02:30:00 = 3600.0
2017-04-16 23:50:00 - 2017-04-16 22:50:00 = 3600.0
2017-04-16 13:00:00 - 2017-04-16 08:32:00 = 16080.0
2017-04-17 19:30:00 - 2017-04-17 13:35:00 = 21300.0
2017-04-17 00:29:00 - 2017-04-16 22:15:00 = 8040.0
2017-04-17 23:00:00 - 2017-04-17 22:25:00 = 2100.0
2017-04-18 18:31:00 - 2017-04-18 18:30:00 = 60.0
2017-04-18 22:00:00 -

2017-05-18 18:07:00 - 2017-05-18 17:55:00 = 720.0
2017-05-18 14:21:00 - 2017-05-18 11:05:00 = 11760.0
2017-05-20 22:25:00 - 2017-05-20 17:36:00 = 17340.0
2017-05-20 18:30:00 - 2017-05-20 18:05:00 = 1500.0
2017-05-20 22:00:00 - 2017-05-20 16:40:00 = 19200.0
2017-05-20 22:15:00 - 2017-05-20 17:31:00 = 17040.0
2017-05-21 03:00:00 - 2017-05-20 22:25:00 = 16500.0
2017-05-22 11:35:00 - 2017-05-22 08:18:00 = 11820.0
2017-05-22 08:00:00 - 2017-05-22 02:20:00 = 20400.0
2017-05-23 19:40:00 - 2017-05-23 15:35:00 = 14700.0
2017-05-23 04:50:00 - 2017-05-23 02:20:00 = 9000.0
2017-05-23 20:35:00 - 2017-05-23 19:01:00 = 5640.0
2017-05-23 12:35:00 - 2017-05-23 12:30:00 = 300.0
2017-05-24 19:40:00 - 2017-05-24 18:05:00 = 5700.0
2017-05-24 05:37:00 - 2017-05-24 02:30:00 = 11220.0
2017-05-24 00:40:00 - 2017-05-23 22:31:00 = 7740.0
2017-05-24 16:40:00 - 2017-05-24 14:37:00 = 7380.0
2017-05-24 16:50:00 - 2017-05-24 12:17:00 = 16380.0
2017-05-24 21:00:00 - 2017-05-24 20:40:00 = 1200.0
2017-05-25 03:20:00 - 2

2017-06-25 09:00:00 - 2017-06-25 03:21:00 = 20340.0
2017-06-25 04:10:00 - 2017-06-25 00:04:00 = 14760.0
2017-06-25 04:10:00 - 2017-06-25 00:04:00 = 14760.0
2017-06-25 04:10:00 - 2017-06-25 00:04:00 = 14760.0
2017-06-25 14:20:00 - 2017-06-25 08:30:00 = 21000.0
2017-06-25 14:20:00 - 2017-06-25 08:25:00 = 21300.0
2017-06-25 03:00:00 - 2017-06-25 00:04:00 = 10560.0
2017-06-25 22:15:00 - 2017-06-25 18:25:00 = 13800.0
2017-06-25 17:19:00 - 2017-06-25 13:45:00 = 12840.0
2017-06-26 12:25:00 - 2017-06-26 07:00:00 = 19500.0
2017-06-26 13:30:00 - 2017-06-26 11:15:00 = 8100.0
2017-06-26 15:00:00 - 2017-06-26 11:15:00 = 13500.0
2017-06-26 21:14:00 - 2017-06-26 19:05:00 = 7740.0
2017-06-27 16:20:00 - 2017-06-27 11:18:00 = 18120.0
2017-06-27 11:25:00 - 2017-06-27 09:08:00 = 8220.0
2017-06-27 14:55:00 - 2017-06-27 09:18:00 = 20220.0
2017-06-27 11:00:00 - 2017-06-27 09:08:00 = 6720.0
2017-06-28 00:15:00 - 2017-06-27 22:00:00 = 8100.0
2017-06-28 00:10:00 - 2017-06-28 00:05:00 = 300.0
2017-06-29 04:00:00

2017-07-30 18:00:00 - 2017-07-30 17:57:00 = 180.0
2017-07-30 12:00:00 - 2017-07-30 10:55:00 = 3900.0
2017-07-31 02:55:00 - 2017-07-31 02:36:00 = 1140.0
2017-07-31 02:55:00 - 2017-07-31 00:00:00 = 10500.0
2017-08-02 20:00:00 - 2017-08-02 19:50:00 = 600.0
2017-08-02 17:58:00 - 2017-08-02 17:50:00 = 480.0
2017-08-02 21:00:00 - 2017-08-02 17:50:00 = 11400.0
2017-08-02 17:30:00 - 2017-08-02 12:26:00 = 18240.0
2017-08-02 17:30:00 - 2017-08-02 12:26:00 = 18240.0
2017-08-03 13:00:00 - 2017-08-03 08:40:00 = 15600.0
2017-08-03 00:10:00 - 2017-08-02 18:57:00 = 18780.0
2017-08-03 00:10:00 - 2017-08-02 19:00:00 = 18600.0
2017-08-03 20:30:00 - 2017-08-03 20:10:00 = 1200.0
2017-08-03 22:00:00 - 2017-08-03 18:51:00 = 11340.0
2017-08-04 19:50:00 - 2017-08-04 19:15:00 = 2100.0
2017-08-04 23:06:00 - 2017-08-04 20:30:00 = 9360.0
2017-08-04 15:35:00 - 2017-08-04 10:30:00 = 18300.0
2017-08-04 23:06:00 - 2017-08-04 20:30:00 = 9360.0
2017-08-04 01:20:00 - 2017-08-03 21:30:00 = 13800.0
2017-08-04 21:00:00 - 20

2017-09-10 23:45:00 - 2017-09-10 19:15:00 = 16200.0
2017-09-10 18:56:00 - 2017-09-10 13:03:00 = 21180.0
2017-09-10 06:00:00 - 2017-09-10 02:10:00 = 13800.0
2017-09-10 00:10:00 - 2017-09-09 23:05:00 = 3900.0
2017-09-11 14:00:00 - 2017-09-11 08:45:00 = 18900.0
2017-09-11 14:00:00 - 2017-09-11 11:50:00 = 7800.0
2017-09-11 16:00:00 - 2017-09-11 12:45:00 = 11700.0
2017-09-11 15:48:00 - 2017-09-11 14:05:00 = 6180.0
2017-09-11 18:34:00 - 2017-09-11 14:05:00 = 16140.0
2017-09-11 00:20:00 - 2017-09-10 19:15:00 = 18300.0
2017-09-12 19:30:00 - 2017-09-12 14:50:00 = 16800.0
2017-09-12 18:12:00 - 2017-09-12 16:30:00 = 6120.0
2017-09-13 17:25:00 - 2017-09-13 12:58:00 = 16020.0
2017-09-13 15:00:00 - 2017-09-13 12:58:00 = 7320.0
2017-09-13 17:40:00 - 2017-09-13 17:30:00 = 600.0
2017-09-13 14:48:00 - 2017-09-13 14:40:00 = 480.0
2017-09-13 16:30:00 - 2017-09-13 14:51:00 = 5940.0
2017-09-13 16:00:00 - 2017-09-13 12:58:00 = 10920.0
2017-09-14 21:55:00 - 2017-09-14 20:05:00 = 6600.0
2017-09-14 22:53:00 - 2

2017-10-15 19:00:00 - 2017-10-15 17:53:00 = 4020.0
2017-10-15 17:43:00 - 2017-10-15 17:40:00 = 180.0
2017-10-16 14:05:00 - 2017-10-16 09:42:00 = 15780.0
2017-10-16 03:20:00 - 2017-10-15 23:17:00 = 14580.0
2017-10-16 01:00:00 - 2017-10-15 22:43:00 = 8220.0
2017-10-17 23:32:00 - 2017-10-17 21:25:00 = 7620.0
2017-10-17 12:13:00 - 2017-10-17 08:08:00 = 14700.0
2017-10-17 20:00:00 - 2017-10-17 18:40:00 = 4800.0
2017-10-17 13:45:00 - 2017-10-17 08:08:00 = 20220.0
2017-10-17 22:20:00 - 2017-10-17 16:42:00 = 20280.0
2017-10-18 13:05:00 - 2017-10-18 13:03:00 = 120.0
2017-10-18 18:15:00 - 2017-10-18 13:50:00 = 15900.0
2017-10-18 17:35:00 - 2017-10-18 16:21:00 = 4440.0
2017-10-19 00:17:00 - 2017-10-19 00:12:00 = 300.0
2017-10-19 22:00:00 - 2017-10-19 21:00:00 = 3600.0
2017-10-19 07:00:00 - 2017-10-19 03:20:00 = 13200.0
2017-10-19 19:16:00 - 2017-10-19 13:21:00 = 21300.0
2017-10-19 23:38:00 - 2017-10-19 19:05:00 = 16380.0
2017-10-19 02:30:00 - 2017-10-19 00:00:00 = 9000.0
2017-10-19 16:35:00 - 201

2017-11-16 08:00:00 - 2017-11-16 03:00:00 = 18000.0
2017-11-16 22:44:00 - 2017-11-16 22:40:00 = 240.0
2017-11-16 08:30:00 - 2017-11-16 08:24:00 = 360.0
2017-11-17 03:52:00 - 2017-11-17 00:52:00 = 10800.0
2017-11-17 00:15:00 - 2017-11-17 00:07:00 = 480.0
2017-11-17 23:53:00 - 2017-11-17 23:50:00 = 180.0
2017-11-17 23:53:00 - 2017-11-17 21:15:00 = 9480.0
2017-11-17 23:50:00 - 2017-11-17 21:15:00 = 9300.0
2017-11-18 23:39:00 - 2017-11-18 19:30:00 = 14940.0
2017-11-18 19:39:00 - 2017-11-18 19:30:00 = 540.0
2017-11-18 23:15:00 - 2017-11-18 19:00:00 = 15300.0
2017-11-18 06:15:00 - 2017-11-18 03:35:00 = 9600.0


In [6]:
crimes['count_ct'] = counts_ct_list
crimes['count_neighbours'] = counts_neighbors_list
crimes['count_total'] = counts_total_list

In [8]:
crimes.to_csv('crimes_pastfrisk_counts.csv')